In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import random

In [ ]:
train_data = pd.read_csv("../input/Kannada-MNIST/train.csv")
test_data = pd.read_csv("../input/Kannada-MNIST/test.csv")
Dig_MNIST = pd.read_csv("../input/Kannada-MNIST/Dig-MNIST.csv")

In [ ]:
train_data.head()

In [ ]:
len(train_data)

In [ ]:
test_data.head()

In [ ]:
len(test_data)

In [ ]:
train_data["label"].unique()

# Check whether data is balanced or not

In [ ]:
total = 0
counter_dict = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0}

for i in train_data["label"]:
    counter_dict[i]+=1
print(counter_dict)

# Data Exploration

In [ ]:
X = train_data.iloc[:, 1:].values
y = train_data.iloc[:, 0].values

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X = X.reshape((X.shape[0], 28, 28, 1))
X.shape

In [ ]:
y

In [ ]:
y = tf.keras.utils.to_categorical(y, num_classes=10)
y.shape

In [ ]:
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid = train_test_split(X, y, test_size = 0.2, random_state=42) 

In [ ]:
def plot_random_digit():
    random_index = np.random.randint(0, X_train.shape[0])
    plt.figure(figsize=(2,2))
    plt.imshow(X_train[random_index], cmap="gray")
    plt.title(np.argmax(y[random_index]))
    plt.axis("Off")
    plt.show()  
plot_random_digit()

In [ ]:
plt.figure(figsize=(12, 8), dpi=200)
for i in range(50):
    plt.subplot(5, 10, i+1)
    index = np.random.randint(0, X_train.shape[0])
    plt.imshow(X_train[index], cmap="gray")
    plt.title(np.argmax(y[index]))
    plt.axis('off')

In [ ]:
learning_rate=0.001
batch_size = 32
epochs = 30
steps_per_epoch = 100
validation_steps = 50

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,BatchNormalization,Conv2D,Dense,Dropout,Flatten,MaxPooling2D

# Set the seed
tf.random.set_seed(42)

# Preprocess data (get all of the pixel values between 1 and 0, also called scaling/normalization)
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow(X_train,Y_train,batch_size=batch_size)
valid_data = valid_datagen.flow(X_valid,Y_valid)

# Create a CNN model (same as Tiny VGG - https://poloclub.github.io/cnn-explainer/)
model_1 = tf.keras.models.Sequential([
    Conv2D(64, 3, padding='same', input_shape=(28, 28, 1)),
    BatchNormalization(scale=False, center=True),
    Activation('relu'),
    Conv2D(64, 3, padding='same'),
    BatchNormalization(scale=False, center=True),
    Activation('relu'),
    MaxPooling2D(2,2),
    Dropout(0.2),

    Conv2D(128, 3, padding='same'),
    BatchNormalization(scale=False, center=True),
    Activation('relu'),
    Conv2D(128, 3, padding='same'),
    BatchNormalization(scale=False, center=True),
    Activation('relu'),
    MaxPooling2D(2,2),
    Dropout(0.2),
    
    Flatten(),
    Dense(512),
    BatchNormalization(scale=False, center=True),
    Activation('relu'),
    Dense(128),
    BatchNormalization(scale=False, center=True),
    Activation('relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model_1.compile(loss=tf.keras.losses.binary_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


# Fit the model

history = model_1.fit(
      train_data,
      steps_per_epoch=steps_per_epoch,
      epochs=epochs,
      validation_data=valid_data,
      validation_steps=validation_steps)

In [ ]:
model_1.summary()

In [ ]:
# Plot the validation and training data separately
def plot_loss_curves(history):
    """
    Returns separate loss curves for training and validation metrics.
    """
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']

    epochs = range(len(history.history['loss']))

    # Plot loss
    plt.plot(epochs, loss, label='training_loss')
    plt.plot(epochs, val_loss, label='val_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()

    # Plot accuracy
    plt.figure()
    plt.plot(epochs, accuracy, label='training_accuracy')
    plt.plot(epochs, val_accuracy, label='val_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend()

In [ ]:
plot_loss_curves(history)

In [ ]:
total = 0
counter_dict = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0}

for i in Dig_MNIST["label"]:
    counter_dict[i]+=1
print(counter_dict)

In [ ]:
# X_extra = extra_data.iloc[:, 1:].values
# X_extra = X_extra.reshape((X_extra.shape[0],28,28,1))
# X_extra.shape
x_dig=Dig_MNIST.drop('label',axis=1).iloc[:,:].values
x_dig = x_dig.reshape(x_dig.shape[0], 28, 28,1)
print(f"x_dig shape: {x_dig.shape}")

In [ ]:
# y_extra = extra_data.label
# y_extra.shape
y_dig=Dig_MNIST.label
print(f"y_dig shape: {y_dig.shape}")

In [ ]:
preds_dig = model_1.predict(x_dig/255)
preds_dig = preds_dig.argmax(axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_dig, preds_dig)
import seaborn as sns
plt.figure(figsize=[7,6])
sns.heatmap(cm, cmap="Reds", annot=True, fmt='.0f')
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_dig, preds_dig))

In [ ]:
sample_submission = pd.read_csv("../input/Kannada-MNIST/sample_submission.csv")

In [ ]:
x_test=test_data.drop('id', axis=1).iloc[:,:].values
x_test = x_test.reshape(x_test.shape[0], 28, 28,1)
print(f"x_test shape: {x_test.shape}")

In [ ]:
predictions = model_1.predict(x_test)
predictions = predictions.argmax(axis=1)

In [ ]:
sample_submission["label"] = predictions

In [ ]:
sample_submission.head()

In [ ]:
sample_submission.to_csv("submission.csv",index=False)